**Лабораторная работа №6** Вариант 9

**Цель лабораторной работы:** научиться решать задачу классификации методами решающих деревьев и ансамблей на их основе.

Выполним анализ набора данных «Анализ текучести кадров в компании». Набор данных содержит информацию о сотрудниках компании, включая их образование, историю работы, демографию и факторы, связанные с занятостью. Необходимо проанализировать проблемы, связанные с текучестью кадров в компании. Набор данных включает следующие атрибуты:


*   **satisfaction_level** – уровень удовлетворенности (0-1)
*   **last_evaluation** - время, прошедшее с момента последней оценки в годах
*   **number_projects** - количество выполненных проектов
*   **average_monthly_hours** - среднее количество рабочих часов за месяц
*   **time_spend_company** - время работы в компании в годах
*   **work_accident** - у сотрудника был несчастный случай на рабочем месте
*   **promotion_last_5years** - было ли у сотрудника повышение за последние 5 лет
*   **sales** - отдел, в котором работает сотрудник
*   **salary** - уровень заработной платы
*   **left** - работник уволился или нет (1 или 0) – целевая переменная


**1. Импорт необходимых библиотек**

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

np.random.seed(42)

**2. Загрузка и первичный анализ данных**

In [ ]:
data = pd.read_csv('/employee.csv')

print("Размерность данных:", data.shape)
print("\nПервые 5 строк данных:")
print(data.head())

print("\nИнформация о данных:")
print(data.info())

print("\nСтатистика данных:")
print(data.describe())

**3. Кодирование категориальных признаков в числовые**

In [ ]:
label_encoder = LabelEncoder()
data['department'] = label_encoder.fit_transform(data['department'])
data['salary'] = label_encoder.fit_transform(data['salary'])
print(data.head())

**4. Разделение данных на признаки и целевую переменную**

In [ ]:
X = data.drop(['left'], axis=1)
y = data['left']

print("Размерность признаков:", X.shape)
print("Названия признаков:", X.columns.tolist())
print("Размерность целевой переменной", y.shape)
print("Уникальные классы:", y.unique())
print("\nПервые 5 строк признаков:")
print(X.head())

**5. Разделение данных на обучающую и тестовую выборки**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"\nОбучающая выборка: {X_train.shape[0]} samples")
print(f"Тестовая выборка: {X_test.shape[0]} samples")

**6. Создание и обучение модели Random Forest**

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
print("\nМодель Random Forest обучена!")

**7. Прогнозирование и оценка модели**

In [ ]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность модели на тестовой выборке: {accuracy:.4f}")

print("\n" + "="*50)
print("Отчет по классификации:")
print("="*50)
print(classification_report(y_test, y_pred))

In [ ]:
# Строим матрицу ошибок
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Матрица ошибок (Confusion Matrix)')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.show()

**8. Анализ важности признаков**

In [ ]:
# Важность признаков на основе среднего уменьшения неоднородности (Gini Importance)
feature_importances = rf_model.feature_importances_
features_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
features_df = features_df.sort_values('importance', ascending=False)

# Визуализация важности признаков
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=features_df.head(15))
plt.title('Топ-15 самых важных признаков (Gini Importance)')
plt.xlabel('Важность')
plt.tight_layout()
plt.show()

In [ ]:
# Выводим таблицу с важностью признаков
print("\nТоп-10 самых важных признаков:")
print(features_df.head(10))

**9. Подбор гиперпараметров**

In [ ]:
from sklearn.model_selection import GridSearchCV

# Определяем сетку параметров для перебора
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Создаем модель для поиска
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Запускаем поиск
grid_search.fit(X_train, y_train)

In [ ]:
# Выводим лучшие параметры
print("\nЛучшие параметры:", grid_search.best_params_)
print("Лучшая точность при кросс-валидации: {:.4f}".format(grid_search.best_score_))

In [ ]:
# Оцениваем лучшую модель на тестовых данных
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(X_test)
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f"Точность улучшенной модели на тестовой выборке: {best_accuracy:.4f}")